In [1]:
import json
import matplotlib.pyplot as plt

from env.chooseenv import make
from run_utils import get_players_and_action_space_list, run_game

In [2]:
game_type = 'snakes_1v1'
game = make(game_type)
game.max_step = 50
log_mode = True
policy_list = ['random'] * len(game.agent_nums)
multi_part_agent_ids, action_spaces = get_players_and_action_space_list(game)
run_game(game, game_type, multi_part_agent_ids, action_spaces, policy_list, log_mode, plot_game=False)

Step10
Step20
Step30
Step40
Step50
